In [26]:
#import ANN_Weather_Forecasting

In [1]:
import pandas as pd  
import numpy as np  
import tensorflow as tf  
from sklearn.metrics import explained_variance_score,mean_absolute_error, median_absolute_error

In [2]:
#import file names
from file_names import file_cleaned, file_test,file_train



#import data

df_weather_train_set = pd.read_csv(file_train)
df_weather_train_set.describe().T
df_weather_train_set.info()
df_weather_train_set_filtered = df_weather_train_set[(df_weather_train_set['STATION_CODES'] == 2)]
df_weather_train_set_filtered=df_weather_train_set_filtered.drop('Unnamed: 0', axis=1)

# SPLIT TESTING AND VALIDATION SET BY SPLITTING LOCATIONS FOR YEARS AFTER 2005
df_weather_test_set = pd.read_csv(file_test)

# ToDo: Not sure whereS the extra 0 is coming from but must remove it
df_weather_test_set=df_weather_test_set.drop('Unnamed: 0', axis=1)
df_weather_test_set_validation = df_weather_test_set[(df_weather_test_set['STATION_CODES'] == 9)]
df_weather_test_set = df_weather_test_set[(df_weather_test_set['STATION_CODES'] == 16)]

frames = [df_weather_train_set_filtered, df_weather_test_set_validation, df_weather_test_set]
df_weather_all = pd.concat(frames)

# print(df_weather_train_set_filtered.shape)
# print(df_weather_test_set.shape)
# print(df_weather_test_set_validation.shape)
# print(df_weather_all.shape)

#combined weather dataset
X = df_weather_all[[col for col in df_weather_all.columns if col != 'AVG_DAILY_TEMP_ALL_HOURS__F']]
y  = df_weather_all['AVG_DAILY_TEMP_ALL_HOURS__F']

# Train, Validate & Test sets

# X will be a pandas dataframe of all columns except AVG_DAILY_TEMP_ALL_HOURS__F
# y will be a pandas series of the AVG_DAILY_TEMP_ALL_HOURS__F
X_val = df_weather_test_set_validation[[col for col in df_weather_test_set_validation.columns if col != 'AVG_DAILY_TEMP_ALL_HOURS__F']]
y_val = df_weather_test_set_validation['AVG_DAILY_TEMP_ALL_HOURS__F']

X_test = df_weather_test_set[[col for col in df_weather_test_set.columns if col != 'AVG_DAILY_TEMP_ALL_HOURS__F']]
y_test = df_weather_test_set['AVG_DAILY_TEMP_ALL_HOURS__F']

X_train= df_weather_train_set_filtered[[col for col in df_weather_train_set_filtered.columns if col != 'AVG_DAILY_TEMP_ALL_HOURS__F']]
y_train = df_weather_train_set_filtered['AVG_DAILY_TEMP_ALL_HOURS__F']


feature_cols = [tf.feature_column.numeric_column(col) for col in X.columns]
# print(feature_cols)


# X_train = X_train.drop(['DEPART_NORMAL__HDDS_FROM_XN__BASE_OF_65_F_2','DEPART_NORMAL__HDDS_FROM_XN__BASE_OF_65_F_3',
#        'DEPART_NORMAL__EDDS_FROM_XN__BASE_OF_65_F_2',
#        'DEPART_NORMAL__EDDS_FROM_XN__BASE_OF_65_F_3',
#        'NORMAL_GROWING_DEGREE_DAYS_XN__BASE_OF_50_F_2',
#        'NORMAL_GROWING_DEGREE_DAYS_XN__BASE_OF_50_F_3', 'COLDEST_WINDCHILL_TEMP__F_2',
#        'COLDEST_WINDCHILL_TEMP__F_3',
#        'AVG_DAILY_TEMP_ALL_HOURS__F_2', 'AVG_DAILY_TEMP_ALL_HOURS__F_3',
#        'NORMAL_AVG_DAILY_WET_BULB_TEMP_ALL_HOURS__F_2',
#        'NORMAL_AVG_DAILY_WET_BULB_TEMP_ALL_HOURS__F_3',
#        'DEPART_NORMAL__CDDS_ALL_HOURS__BASE_OF_65_F_2',
#        'DEPART_NORMAL__CDDS_ALL_HOURS__BASE_OF_65_F_3', 
#        'HIGH_TEMP_F_2', 'HIGH_TEMP_F_3',
#        'DEPART_NORMAL__EDDS_ALL_HOURS__BASE_OF_65_F_2',
#        'DEPART_NORMAL__EDDS_ALL_HOURS__BASE_OF_65_F_3',
#         'DFN_AVG_DAILY_TEMP_XN__F_2',
#        'DFN_AVG_DAILY_TEMP_XN__F_3'] , axis=1)

# X_test = X_test.drop(['DEPART_NORMAL__HDDS_FROM_XN__BASE_OF_65_F_2','DEPART_NORMAL__HDDS_FROM_XN__BASE_OF_65_F_3',
#        'DEPART_NORMAL__EDDS_FROM_XN__BASE_OF_65_F_2',
#        'DEPART_NORMAL__EDDS_FROM_XN__BASE_OF_65_F_3',
#        'NORMAL_GROWING_DEGREE_DAYS_XN__BASE_OF_50_F_2',
#        'NORMAL_GROWING_DEGREE_DAYS_XN__BASE_OF_50_F_3', 'COLDEST_WINDCHILL_TEMP__F_2',
#        'COLDEST_WINDCHILL_TEMP__F_3',
#        'AVG_DAILY_TEMP_ALL_HOURS__F_2', 'AVG_DAILY_TEMP_ALL_HOURS__F_3',
#        'NORMAL_AVG_DAILY_WET_BULB_TEMP_ALL_HOURS__F_2',
#        'NORMAL_AVG_DAILY_WET_BULB_TEMP_ALL_HOURS__F_3',
#        'DEPART_NORMAL__CDDS_ALL_HOURS__BASE_OF_65_F_2',
#        'DEPART_NORMAL__CDDS_ALL_HOURS__BASE_OF_65_F_3', 
#        'HIGH_TEMP_F_2', 'HIGH_TEMP_F_3',
#        'DEPART_NORMAL__EDDS_ALL_HOURS__BASE_OF_65_F_2',
#        'DEPART_NORMAL__EDDS_ALL_HOURS__BASE_OF_65_F_3',
#         'DFN_AVG_DAILY_TEMP_XN__F_2',
#        'DFN_AVG_DAILY_TEMP_XN__F_3'] , axis=1)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39438 entries, 0 to 39437
Data columns (total 33 columns):
Unnamed: 0                                       39438 non-null int64
AVG_DAILY_TEMP_ALL_HOURS__F                      39438 non-null float64
STATION_CODES                                    39438 non-null int64
DEPART_NORMAL__HDDS_FROM_XN__BASE_OF_65_F_1      39437 non-null float64
DEPART_NORMAL__HDDS_FROM_XN__BASE_OF_65_F_2      39436 non-null float64
DEPART_NORMAL__HDDS_FROM_XN__BASE_OF_65_F_3      39435 non-null float64
DEPART_NORMAL__EDDS_FROM_XN__BASE_OF_65_F_1      39437 non-null float64
DEPART_NORMAL__EDDS_FROM_XN__BASE_OF_65_F_2      39436 non-null float64
DEPART_NORMAL__EDDS_FROM_XN__BASE_OF_65_F_3      39435 non-null float64
NORMAL_GROWING_DEGREE_DAYS_XN__BASE_OF_50_F_1    39437 non-null float64
NORMAL_GROWING_DEGREE_DAYS_XN__BASE_OF_50_F_2    39436 non-null float64
NORMAL_GROWING_DEGREE_DAYS_XN__BASE_OF_50_F_3    39435 non-null float64
COLDEST_WINDCHILL_TEMP__F_1    

In [3]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

print(X_train.head())

X_test.shape

(2191, 31)
(2191,)
(731, 31)
(731,)
      STATION_CODES  DEPART_NORMAL__HDDS_FROM_XN__BASE_OF_65_F_1  \
4382              2                                          6.5   
4383              2                                        -10.4   
4384              2                                         -4.3   
4385              2                                         -7.7   
4386              2                                          4.0   

      DEPART_NORMAL__HDDS_FROM_XN__BASE_OF_65_F_2  \
4382                                          5.0   
4383                                          6.5   
4384                                        -10.4   
4385                                         -4.3   
4386                                         -7.7   

      DEPART_NORMAL__HDDS_FROM_XN__BASE_OF_65_F_3  \
4382                                          1.4   
4383                                          5.0   
4384                                          6.5   
4385                    

(731, 31)

In [8]:
regressor = tf.estimator.DNNRegressor(feature_columns=feature_cols,
                                      hidden_units=[50,50],
                                      model_dir='tf_wx_model_v3')

def wx_input_fn(X, y=None, num_epochs=None, shuffle=True, batch_size=2400):
    return tf.estimator.inputs.pandas_input_fn(x=,
                                               y=,
                                               num_epochs=num_epochs,
                                               shuffle=shuffle,
                                               batch_size=batch_size)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'tf_wx_model_v3', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a3987e4e0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
evaluations = []
STEPS = 2300
for i in range(100):
    regressor.train(input_fn=wx_input_fn(X_train, y=y_train), steps=STEPS)
    evaluations.append(regressor.evaluate(input_fn=wx_input_fn(X_val,
                                                               y_val,
                                                               num_epochs=1,
                                                               shuffle=False)))

In [ ]:
import matplotlib.pyplot as plt  
%matplotlib inline

# manually set the parameters of the figure to and appropriate size
plt.rcParams['figure.figsize'] = [14, 10]

loss_values = [ev['loss'] for ev in evaluations]  
training_steps = [ev['global_step'] for ev in evaluations]

plt.scatter(x=training_steps, y=loss_values)  
plt.xlabel('Training steps (Epochs = steps / 2)')  
plt.ylabel('Loss (SSE)')  
plt.show()  

In [16]:
pred = regressor.predict(input_fn=wx_input_fn(X_val,  
                                              num_epochs=1,
                                              shuffle=False))
predictions = np.array([p['predictions'][0] for p in pred])

print("The Explained Variance: %.2f" % explained_variance_score(  
                                            y_val, predictions))  
# print("The Mean Absolute Error: %.2f degrees Celcius" % mean_absolute_error(  
#                                             y_val, predictions))  
# print("The Median Absolute Error: %.2f degrees Celcius" % median_absolute_error(  
#                                             y_val, predictions))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from tf_wx_model_v3/model.ckpt-101200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


ValueError: Found input variables with inconsistent numbers of samples: [731, 2191]

In [28]:
#y_val.shape
predictions[0].shape

()

In [18]:
predictions.shape

(2191,)